In [23]:
re.sub(r'\ba\b','one','this is a pen')

'this is one pen'

In [53]:
from __future__ import print_function
from sklearn.feature_extraction.text import CountVectorizer
import os
from gensim.models import Word2Vec
import re
import numpy as np
#this is a different euclidean distances from the model
from sklearn.metrics.pairwise import euclidean_distances as pair_euclidean_distances
from sklearn.metrics import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from pyemd import emd

In [46]:
data_path = '/Users/Rutherford/Desktop/data'
_save_path = os.path.join(data_path,'model.ckpt')
epsilon = 1e-4

binary_file = os.path.join(data_path,
                           'GoogleNews-vectors-negative300.bin')
w2v_dat = os.path.join(data_path,'embed.dat')
w2v_vocab = os.path.join(data_path,'embed.vocab')

if not os.path.exists(w2v_dat):
    print("Caching word embeddings in memmapped format.                     Please be patient...")
    wv = Word2Vec.load_word2vec_format(
        binary_file,binary=True)
    fp = np.memmap(w2v_dat, dtype=np.double,
                   mode='w+', shape=wv.syn0.shape)
    fp[:] = wv.syn0[:]
    with open(w2v_vocab, "w") as f:
        for _, w in sorted((voc.index, word)                                    for word, voc in wv.vocab.items()):
            print(w, file=f)
    del fp, wv

# create word embeddings and mapping of vocabulary item to index
embeddings = np.memmap(w2v_dat, dtype=np.float64,
                            mode="r", shape=(3000000, 300))
with open(w2v_vocab) as f:
    vocab_list = map(lambda string: string.strip(), f.readlines())
vocab_dict = {w: i for i, w in enumerate(vocab_list)}

# mean of 20 rarest words, used as a stand-in for pairwise distances
# if a word is out-of-vocabulary
avg_rare_word = np.mean(np.vstack((embeddings[-20:])),axis=0)

In [47]:
s_1 = 'this is a pen'
s_2 = 'wishes to men'

In [106]:
vect = CountVectorizer(token_pattern='[\w\']+').fit([s_1, s_2])
features = np.asarray(vect.get_feature_names())

# get 'flow' vectors
v_1, v_2 = vect.transform([s_1, s_2])
v_1 = v_1.toarray().ravel().astype(np.float64)
v_2 = v_2.toarray().ravel().astype(np.float64)

# normalize vectors so as not to reward shorter strings in WMD
v_1 /= (v_1.sum()+epsilon)
v_2 /= (v_2.sum()+epsilon)

# for each out-of-vocabulary item, use the average of the 20
# rarest words' embeddings to represent it in the distance calc
bad = len([w for w in features if w not in vocab_dict])
bad_rows = np.asarray([avg_rare_word]*bad)

# get distance matrix for words in both strings
W_ = embeddings[[vocab_dict[w] for w in features if w in vocab_dict]]

if bad_rows.shape[0]>0:
    W_ = np.vstack((W_,bad_rows))

# use both euclidean and cosine dists (cosine dist is 1-cosine sim)
D_pair_euclidean = pair_euclidean_distances(W_).astype(np.float64)
D_euclidean = euclidean_distances(W_).astype(np.float64)
D_cosine = 1.-cosine_similarity(W_,).astype(np.float64)

# using EMD (Earth Mover's Distance) from PyEMD
distances_pair_euclidean = emd(v_1,v_2,D_pair_euclidean)
distances_euclidean = emd(v_1,v_2,D_euclidean)
distances_cosine = emd(v_1,v_2,D_cosine)


In [126]:
vect = CountVectorizer(token_pattern='[\w\']+').fit([s_1, s_2])
features = np.asarray(vect.get_feature_names())

# get 'flow' vectors
v_1, v_2 = vect.transform([s_1, s_2])
v_1 = v_1.toarray().ravel().astype(np.float64)
v_2 = v_2.toarray().ravel().astype(np.float64)

# normalize vectors so as not to reward shorter strings in WMD
v_1 /= (v_1.sum()+epsilon)
v_2 /= (v_2.sum()+epsilon)

bad_row = np.asarray([avg_rare_word])

W_ = np.ndarray([0,300])
# get distance matrix for words in both strings
for w in features:
    if w in vocab_dict:
        #print(embeddings[vocab_dict[w]].shape)
        print(embeddings[[vocab_dict[w]]].shape)
        
        W_ = np.append(W_,embeddings[[vocab_dict[w]]],axis=0)
    else:
        print(bad_row.shape)
        W_ = np.append(W_,bad_row,axis=0)

# use both euclidean and cosine dists (cosine dist is 1-cosine sim)
D_pair_euclidean = pair_euclidean_distances(W_).astype(np.float64)
D_euclidean = euclidean_distances(W_).astype(np.float64)
D_cosine = 1.-cosine_similarity(W_,).astype(np.float64)

# using EMD (Earth Mover's Distance) from PyEMD
distances_pair_euclidean = emd(v_1,v_2,D_pair_euclidean)
distances_euclidean = emd(v_1,v_2,D_euclidean)
distances_cosine = emd(v_1,v_2,D_cosine)

(1, 300)
(1, 300)
(1, 300)
(1, 300)
(1, 300)
(1, 300)
(1, 300)


In [105]:
print(distances_pair_euclidean)
print(distances_euclidean)
print(distances_cosine)

2.34285048785
2.34285048785
0.68416614785


In [127]:
W_ = np.ndarray([0,300])
# get distance matrix for words in both strings
for w in features:
    if w in vocab_dict:
        W_ = np.append(W_,embeddings[[vocab_dict[w]]],axis=0)
    else:
        W_ = np.append(W_,bad_row,axis=0)
